In [18]:
# import nltk
# from nltk import pos_tag, word_tokenize
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('gutenberg')
# import re
# import string
# from itertools import combinations
# from collections import Counter


from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
from flair.models import SequenceTagger
from flair.data import Sentence
import operator

from nervaluate import Evaluator
from nltk.tokenize import sent_tokenize


In [19]:
# Use flair named entity recognition
tagger = SequenceTagger.load('ner')     # ner, ner-ontonotes, ...

2022-05-24 17:02:31,528 loading file /Users/matijagercer/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-05-24 17:02:35,741 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [34]:
def read_text(path):
    with open(path, encoding="utf-8") as f:
        text = f.read()
        text = text.replace('\r', ' ').replace('\n', ' ')\
            .replace("’", "'").replace("\"", "").replace("”", "").replace("“", "")
    return text

def flair_NER(book, eval = False):
    """
    flair_NER vrne seznam, v katerem so shranjene prepoznane identitete glede na posamezni stavek
    :param book: str
    :return: entity_dict (seznam slovarjev kot npr. [name, tag, start_pos, stop_pos, line_num. token_num])
    """
    
    # EVAL ---
    if eval == True:
        sentences, sentences_tag = conll_read()
        #print(len(sentences), "--------")
    else:
        sentences = sent_tokenize(book)
    sentences_NER = []
    sentences_NER_results = []
    # EVAL ---

    
    
    # 01 NER model
    entity_dict = []
    for line_num, line in enumerate(tqdm(sentences)):
        
        
        
        # EVAL ---
        words_list = word_tokenize(line)
        tags_list = ["O"] * len(words_list)
        sentences_NER.append(words_list)
        # EVAL ---
        
        

        # 01a Predict:
        sentence = Sentence(line)
        tagger.predict(sentence)

        # 01b Vmesni izpis (brez lokacije):
        #print(sentence.to_tagged_string())
        #print(sentence.get_spans('ner'))

        # 01c Ekstrakcija podatka iz stavka:
        for entity in sentence.get_spans('ner'):        # veliko različnih flair - nerov
            name = entity.text

            # str location
            # start_pos = entity.start_position    # št. str
            # stop_pos = entity.end_position      # št. str

            # token location
            tmp_flag = True
            for token in entity:
                if tmp_flag:
                    start_pos = token.idx -1
                    stop_pos = token.idx
                else:
                    stop_pos = token.idx
                tmp_flag = False

            tag = entity.get_label("ner").value             # tag = entity.tag
            conf_score = entity.get_label("ner").score      # conf_score = entity.score

            info_dict = {}
            info_dict["name"] = name
            info_dict["tag"] = tag
            info_dict["start_pos"] = start_pos
            info_dict["stop_pos"] = stop_pos
            info_dict["line_num"] = line_num

            entity_dict.append(info_dict)
            
            
            
            # EVAL: ---
            names = name.split(" ")
            ff_flag = False
            if tag == "PER":
                for name in names:
                    try:
                        if ff_flag == False:
                            idx = words_list.index(name)
                            tags_list[idx] = "B-PER"
                            ff_flag = True
                        else:
                            idx = words_list.index(name)
                            tags_list[idx] = "I-PER"
                    except:
                        print("Not found: ", name)
            # EVAL ---
            
        sentences_NER_results.append(tags_list)
            
    return entity_dict, sentences_NER, sentences_NER_results



def get_names_from_NER(entity_dict):
    """
    get_names_from_NER sprejme entity_dict in vrne urejen seznam terk ("ime", št_zaznano)
    :param entity_dict: dict (seznam dictov)
    :return: unique_names: list
    """
    unique_names = {}

    for entity in entity_dict:
        if entity["tag"] == "PER":
            if entity["name"] not in unique_names:
                unique_names[entity["name"]] = 1
            else:
                unique_names[entity["name"]] += 1
    unique_names = sorted(unique_names.items(), key=operator.itemgetter(1),reverse=True)

    return unique_names


def conll_read():
    with open("conll2003/valid.txt") as f:
        lines = f.readlines()
    sentences = []
    sentences_tag = []
    tmp = ""
    tmp_tag = ""
    tags = []
    for i in range(len(lines)):

        line = lines[i]
        line = line.split(" ")
        text_word = line[0]

        if text_word[0] != "-":
            if text_word == "\n":
                # print(tmp)
                # print(tmp_tag)
                # print("-------")
                sentences.append(tmp)
                tmp = ""
                #sentences_tag.append(tmp_tag)
                #tmp_tag = ""
                sentences_tag.append(tags)
                tags = []

            elif text_word == "(" or text_word == ")" or text_word == '"' or text_word == ':':
                pass
            else:
                if text_word[0] != "." and text_word[0] != "," and text_word[0] != ":" and text_word[0] != ";"\
                        and text_word[0] != "!" and text_word[0] != "?" and text_word[0] != ")" and text_word[0] != "'":   #or text_word[0] != "," or text_word[0] != "(" or text_word[0] != ")":
                    tmp = tmp + " " + text_word
                    line[3] = line[3].replace("\n", "")

                    if line[3] == "B-PER" or line[3] == "I-PER":
                        #line[3] = "B-PER"
                        #tmp_tag = tmp_tag + " " + line[3]
                        tags.append(line[3])
                    else:
                        line[3] = "O"
                        #tmp_tag = tmp_tag + " " + line[3]
                        tags.append(line[3])
                else:
                    tmp = tmp +  text_word
                    line[3] = "O"
                    tags.append(line[3])
    return sentences, sentences_tag

In [40]:
book = read_text('../../data/books/ASongOfIceAndFire/AGOT/chapters/1_Bran_1.txt')
entity_dict, sentences_NER, sentences_NER_results = flair_NER(book, eval= False)
unique_names = get_names_from_NER(entity_dict)

for (name, num) in unique_names:
    if True: #num > 1:
        print(name, num)
        


100%|█████████████████████████████████████████████████████████████| 284/284 [02:38<00:00,  1.79it/s]

Bran 46
Robb 29
Jon 28
Jory 9
Greyjoy 7
Theon Greyjoy 6
Father 5
Hullen 5
Jon Snow 3
Old Nan 2
Lord Stark 2
Jory Cassel 2
House Stark 2
Stark 2
Mance Rayder 1
Eddard Stark 1
Robert of the House Baratheon 1
Eddard 1
Lord of Winterfell 1
Theon 1
Robert 1
Starks 1
Gods 1
Harwin 1
Ser Rodrik 1
Rickon 1
Snow 1
Desmond 1


In [36]:
# Evalvacija na datasetu Conll2003 (osredotočena na imena)

def eval_wrapper():
    entity_dict, sentences_NER, sentences_NER_results = flair_NER("", eval= True)
    unique_names = get_names_from_NER(entity_dict)

    # Kontrola podatkov
    sentences_org, sentences_org_tag = conll_read()
    # for x in range(len(sentences_NER)):
    #     print("-----------", x)
    #     #print(len(sentences_org_tag[x]))
    #     print(len(sentences_NER[x]))
    #     print(len(sentences_NER_results[x]))

    #     #print(sentences_org[x])    # 22 26
    #     print(sentences_NER[x])
    #     print(sentences_org_tag[x])
    #     print(sentences_NER_results[x])
    
    true = sentences_org_tag
    pred = sentences_NER_results
    evaluator = Evaluator(true, pred, tags=['PER'], loader="list")

    results, results_by_tag = evaluator.evaluate()
    print(results)
    

In [73]:
## Evalvacija za I. poglavje GoT

from chapter__1annotated import chapter1_lst

# Kontrola:
# print(len(sentences_NER_results))
# print(len(chapter1_lst))

# for x in range(len(sentences_NER)):
#     print("-----------", x)
#     print(len(sentences_NER[x]))
#     print(len(sentences_NER_results[x]))
#     print(len(chapter1_lst[x]))
    

#     print(sentences_NER[x])
#     print(sentences_NER_results[x])
#     print(chapter1_lst[x])

In [74]:
true = chapter1_lst
pred = sentences_NER_results
evaluator = Evaluator(true, pred, tags=['PER'], loader="list")

results, results_by_tag = evaluator.evaluate()
print(results)

{'ent_type': {'correct': 156, 'incorrect': 0, 'partial': 0, 'missed': 8, 'spurious': 7, 'possible': 164, 'actual': 163, 'precision': 0.9570552147239264, 'recall': 0.9512195121951219, 'f1': 0.9541284403669725}, 'partial': {'correct': 155, 'incorrect': 0, 'partial': 1, 'missed': 8, 'spurious': 7, 'possible': 164, 'actual': 163, 'precision': 0.9539877300613497, 'recall': 0.948170731707317, 'f1': 0.9510703363914372}, 'strict': {'correct': 155, 'incorrect': 1, 'partial': 0, 'missed': 8, 'spurious': 7, 'possible': 164, 'actual': 163, 'precision': 0.950920245398773, 'recall': 0.9451219512195121, 'f1': 0.948012232415902}, 'exact': {'correct': 155, 'incorrect': 1, 'partial': 0, 'missed': 8, 'spurious': 7, 'possible': 164, 'actual': 163, 'precision': 0.950920245398773, 'recall': 0.9451219512195121, 'f1': 0.948012232415902}}
